Nombre del estudiante: María José Belda Beneyto


# Tratamiendo de Datos Masivos - Práctica 1

Queremos descargar información sobre incidencias de circulación a partir la página 

http://infocar.dgt.es/etraffic/Incidencias?caracter=acontecimiento&orden=fechahora_ini%20DESC

Lo primero es asegurarnos de que disponemos de selenium y de [chromedriver](https://sites.google.com/a/chromium.org/chromedriver/home). En la siguiente caja de código debemos poner el path a chromedriver

In [1]:
chromedriver = "c:/hlocal/tdm/chromedriver.exe"


La siguiente caja de código debe abrir un navegador chrome y deja en la variable `driver` el enlace al navegador que utilizaremos de punto de partida para obtener los datos.

In [4]:
# Arrancamos una instancia de google Chrome. 
import os
from selenium import webdriver  # si da error, desde anaconda prompt sobre python 3.x hacer pip install --user  selenium

os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(executable_path=chromedriver,options=chrome_options)

#### Ejercicio 1 [0.5]

Cargar la página indicada en la URL

In [5]:
url = "http://infocar.dgt.es/etraffic/Incidencias?caracter=acontecimiento&orden=fechahora_ini%20DESC"

Solución

In [6]:
driver.get(url)

#### Ejercicio 2 [1]
Seleccionar la comunidad autónoma de Madrid de la lista de comunidades. 

Pista:
- Primero inspeccionar la lista para ver cómo acceder a ella. Normalmente utilizaremos alguna variante de find_element_xxxx. La lista de estos métodos puede consultarse [aquí](https://selenium-python.readthedocs.io/locating-elements.html)

- Una vez hecho esto investigar el uso de Select para seleccionar el elemento que deseemos de la lista. 

Ojo: si se usa el texto de la lista desplegable para seleccionar, ser cuidadoso con poner el texto tal cual aparece


Solución:

In [8]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_id('ca'))
select.select_by_visible_text('Madrid, Comunidad de ')

Análogamente, seleccionar la provincia de Madrid de la lista de provincias.

Solución:

In [9]:
select = Select(driver.find_element_by_id('provIci'))
select.select_by_visible_text('MADRID')

#### Ejercicio 3 [1]

Hacer click en las casillas (check box) "Retenciones" y "Obras"

Solución:

In [13]:
driver.find_element_by_id('IncidenciasRETENCION').click()
driver.find_element_by_id('IncidenciasOBRAS').click()

#### Ejercicio 4 [0.5]

Hacemos click en el botón "Consultar"

In [14]:
driver.find_element_by_id('consultar').click()

A partir de aquí nos centraremos en  la información que se muestra por pantalla, para extraerla y analizarla
![](https://i.ibb.co/R0FMzyr/tabla.png)

#### Ejercicio 5 [1 punto]

¿Se te ocurre, para el futuro, alguna forma más rápida de acceder a esta página sin tener que seleccionar comunidad y provincia de listas y marcar las casillas Retenciones y Obras ?

Solucíon (escribir un par de líneas explicando la propuesta)

In [ ]:
Estaría bien que se añadiese a la web un acceso directo para descargar los datos en formato csv como ya existe en otras webs.
De esa forma, el fichero sería más grande porque contendría los datos de todas las provincias y comunidades pero se podrían
manejar más fácilmente con un data frame.

#### Ejercicio 6  [1 punto]

La primera palabra de la primera línea de la columna DESCRIPCIÓN será OBRA si se trata de una obra o RETENCIÓN si es una retención. Mostrar el número de obras. 

Nota: se valorará (medio punto) el uso de `find_elements_by_xpath` para obtener la lista de elementos tipo td que contienen el texto que puede comenzar (o no) con "OBRA".

Solución:

In [36]:
tabla = driver.find_element_by_id('tablaincidencias')
trs = tabla.find_elements_by_xpath("tbody/tr")

#Recorremos las filas, leyendo el valor de la columna descripción y viendo si es una obra
obras = [] #Me guardo las incidencias que son obras para la última pregunta
i = 0
numObras = 0
while i<len(trs): 
    tr = trs[i]
    desc = tr.find_element_by_xpath("td[6]").text
    if "OBRA" in desc:
        numObras+=1
        obras.append(tr)
    i+=1

numObras

33

#### Ejercicio 7 [2 puntos]
para cada incidencia cuya fecha de inicio (primera columna)  no coincida con el día de la fecha, mostrar su fecha de inicio (solo fecha, la hora no importa). Al acabar mostrar el total de incidencias con estas características.

Solución:

In [34]:
#Primero conseguimos la fecha actual
from datetime import date

today = date.today()
# dd/mm/yyyy
dtoday = today.strftime("%d/%m/%Y")

#Ahora conseguimos las fechas de las incidencias
i = 0
cont = 0
while i<len(trs): 
    tr = trs[i]
    col1 = tr.find_element_by_xpath("td[1]")
    fecha = col1.find_element_by_class_name("p2TablaIncidencias").text
    if fecha != dtoday:
        print(fecha)
        cont+=1
    i+=1

print(cont)


02/10/2020
02/10/2020
10/09/2020
10/09/2020
10/09/2020
07/08/2020
07/08/2020
21/06/2020
05/08/2019
9


#### Ejercicio 8 [3 puntos]
Nos interesa el número de habitantes de los municipios con obras (segunda fila dentro de la columna PROVINCIA/POBLACIÓN). Para ello podemos consultar un fichero de municipios de la comunidad de madrid:

https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/municipiosmadrid.csv

que tiene estos datos. Es un fichero CSV con cabecera, en particular la columna `municipio` tiene el nombre del muncipio, mientras que la columna `total`  contiene el total. Recorrer la lista de minicipios en la tabla de tráfico (no importa si se trata de incidencias u obras) y mostrar el nombre junto con su número de habitantes. Se valorará (0.5 puntos) qno mostrar el mismo municipio más de una vez. 

Nota: en ocasiones la línea viene en blanco, o lo que viene no es un municipiol, o vienen varios...en esos casos simplemente la cadena no se encontrará en la primera columna del fichero de municipios, no tenemos que hacer nada.

Solución:

In [50]:
#Importamos los datos de habitantes de cada municipio
import pandas as pd
valores = "https://raw.githubusercontent.com/RafaelCaballero/tdm/master/datos/municipiosmadrid.csv"
df = pd.read_csv(valores,encoding= 'unicode_escape')

#Conseguimos los nombres de los municipios
i = 0
cont = 0
nombresMuni = []
while i<len(obras): 
    tr = obras[i]
    col1 = tr.find_element_by_xpath("td[4]")
    municipio = col1.find_element_by_class_name("p2TablaIncidencias").text
    if municipio not in nombresMuni:
        nombresMuni.append(municipio)
    i+=1

nombresMuni.sort()

#Recorremos los datos de la tabla y para los municipios que tienen obras mostramos su nombre y sus habitantes 
for index, r in df.iterrows():
    if r.municipio in nombresMuni:
        print(r.municipio + " " + str(r.total))

ARANJUEZ 59607
BOADILLA DEL MONTE 54570
CENICIENTOS 2033
CERCEDILLA 7026
COSLADA 81661
MADRID 3266126
MECO 14305
NAVAS DEL REY 2918
RASCAFRIA 1664
ROBREGORDO 58
VELILLA DE SAN ANTONIO 12236
VILLAREJO DE SALVANES 7335


In [52]:
driver.close()